## **Hostear API en Azure**

Se ha tomado como referencia la documentación de microsoft [microsoft](https://learn.microsoft.com/en-us/azure/developer/python/tutorial-containerize-simple-web-app-for-app-service?tabs=web-app-flask), la cual nos indica cómo desplegar una aplicación web de Flask o FastAPI en contenedores en Azure App Service.

Este tutorial te muestra cómo desplegar una aplicación web de Flask o FastAPI en Azure App Service utilizando la característica de Web App for Containers. Web App for Containers proporciona una vía de acceso fácil para que los desarrolladores aprovechen la plataforma de Azure App Service completamente administrada, pero que también desean un único artefacto desplegable que contenga una aplicación y todas sus dependencias. Para obtener más información sobre el uso de contenedores en Azure, consulta Comparando opciones de contenedores de Azure.


Prerrequisitos:

 * Una cuenta de Azure donde puedas desplegar una aplicación web en Azure App Service y Azure Container Registry.
 * Azure CLI para crear una imagen Docker y desplegarla en App Service. Y opcionalmente, Docker y el Docker CLI para crear y probar una imagen Docker en tu entorno local.


 
En este caso el primer paso era posible obviarlo, dado que ya nuestro repo se encontrab clonado, sin embargo es posible realizar el clone del repo donde se encuentra contenida la API

```bash
git clone https://github.com/jmpaiba123/AB_Testing_Meli.git
```

### **Agregar los archivos Dockerfile y .dockerignore**

Agrega un Dockerfile para instruir a Docker cómo construir la imagen. El Dockerfile especifica el uso de Gunicorn, un servidor web de nivel de producción que reenvía las solicitudes web a los frameworks Flask y FastAPI. Los comandos ENTRYPOINT y CMD instruyen a Gunicorn a manejar las solicitudes para el objeto app. 50505 se utiliza para el puerto del contenedor (interno) en este ejemplo, pero puedes usar cualquier puerto libre.


```dockerfile
# syntax=docker/dockerfile:
FROM python:3.11

WORKDIR /code

COPY requirements.txt .

RUN pip3 install -r requirements.txt

COPY . .

EXPOSE 50505

ENTRYPOINT ["gunicorn", "app:app"]
```

En nuestro caso adicional, es necesario asegurarnos que nuestro archivo **requirements.txt** contenga gunicorn y pandas.

```python
Flask==2.2.2
gunicorn
Werkzeug==2.2.2
pandas
```

y agregamos un archivo .dockerignore para excluir archivos innecesarios de la imagen.

```dockerignore
.git*
**/*.pyc
.venv/
```


###  **Archivo de configuración de Gunicorn**

Gunicorn puede configurarse con un archivo gunicorn.conf.py. Cuando el archivo gunicorn.conf.py se encuentra en el mismo directorio donde se ejecuta Gunicorn, no necesitas especificar su ubicación en el Dockerfile.

```python
import multiprocessing

max_requests = 1000
max_requests_jitter = 50

log_file = "-"

bind = "0.0.0.0:50505"

workers = (multiprocessing.cpu_count() * 2) + 1
threads = workers

timeout = 120
```


###  **Construir y ejecutar la imagen localmente**


Si el comando docker build devuelve un error, asegúrate de que el demonio Docker esté ejecutándose. En Windows, asegúrate de que Docker Desktop esté ejecutándose.Ejecuta la imagen localmente en un contenedor Docker.

```bash

docker build --tag pruebamelijmpaiba
docker run --detach --publish 5000:50505 pruebamelijmpaiba

```
Abre la URL http://localhost:5000 en tu navegador para ver la aplicación web ejecutándose localmente.



### **Crear un grupo de recursos y un registro de contenedores en Azure**

Crea un grupo con el comando az group create. Crea un Azure Container Registry con el comando az acr create. Y Establece una variable de entorno con el valor de la contraseña para el registro.

```bash

az group create --name web-app-simple-rg --location eastus
az acr create --resource-group web-app-simple-rg --name pruebamelijmpaiba --sku Basic --admin-enabled true
ACR_PASSWORD=$(az acr credential show --resource-group web-app-simple-rg --name pruebamelijmpaiba --query "passwords[?name == 'password'].value" --output tsv)

```


### **Construye la imagen Docker en Azure con el comando az acr build y Desplegar la aplicación web en Azure.**

Construye la imagen Docker en Azure con el comando az acr build, Crea un plan de App Service con el comando az appservice plan create. Crea la aplicación web con el comando az webapp create.

```bash

az acr build --resource-group web-app-simple-rg --registry pruebamelijmpaiba --image pruebamelijmpaiba:latest 
az appservice plan create --name webplan --resource-group web-app-simple-rg --sku B1 --is-linux
az webapp create --resource-group web-app-simple-rg --plan webplan --name pruebamelijmpaiba --docker-registry-server-password $ACR_PASSWORD --docker-registry-server-user pruebamelijmpaiba --role acrpull --deployment-container-image-name pruebamelijmpaiba.azurecr.io/pruebamelijmpaiba:latest

```

### **Actualizar y volver a desplegar**

Después de hacer cambios en el código, puedes volver a desplegar en App Service con los comandos az acr build y az webapp update.

```bash

az acr build --resource-group web-app-simple-rg --registry pruebamelijmpaiba --image pruebamelijmpaiba:latest .
az webapp update --resource-group web-app-simple-rg --name pruebamelijmpaiba --docker-registry-server-url pruebamelijmpaiba.azurecr.io --docker-custom-image-name pruebamelijmpaiba:latest

```

### **Limpiar recursos**

Para eliminar los recursos, usa el comando az group delete.

```bash

az group delete --name web-app-simple-rg

```

También puedes eliminar el grupo en el portal de Azure o en Visual Studio Code con la extensión Azure Tools. Siguiendo estos pasos y utilizando estos archivos, fue posible poder desplegar nuestra API Flask en Azure App Service utilizando Docker y Azure Container Registry.

### **Consulta de nuestra API desde el host de azure:** http://pruebamelijmpaiba.azurewebsites.net/experiment/sa-on-vip/result?day=2021-08-01